# Extract edges that are in a cluster

In [ ]:
# Input edge_list of 3 columns: node1, node2, weight. No header.
edge_df = pd.read_csv('name_edgelist.tsv', sep="\t", header=None)
# ClusterONE output results
dfone = pd.read_csv('name_clusterONE.csv')
# Convert the node column of each cluster to a list of members
dfone["Members"] = dfone["Members"].apply(lambda x: x.split(" "))

In [ ]:
from multiprocessing import  Pool

def is_in_cluster(splt):
    # A function that keeps edges only if they are found within a ClusterONE cluster
    return(splt[splt[[0,1]].apply(lambda a: 
        any(dfone["Members"].apply(lambda x: set([a[0], a[1],]).issubset(x))), axis=1)])

#edges_in_clusters = test_df[test_df[[0,1]].apply(is_in_cluster, axis=1)]

def parallel_is_in_cluster(df, func, n_cores=32):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

edges_in_clusters_df = parallel_is_in_cluster(edge_df, is_in_cluster)
print(len(edges_in_clusters_df))
edges_in_clusters_df